## Import libraries

In [ ]:
import numpy as np
from PIL import Image
from collections import defaultdict
from google.colab import drive
import pandas as pd
import os.path
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import accuracy_score,roc_auc_score
import matplotlib as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from scipy.stats import mode
from xgboost import XGBClassifier

##google drive

In [ ]:
PROJECT_PATH= '/content/drive/MyDrive/project/'

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Constants

In [ ]:
DATA='NSL-KDD'

# function

In [ ]:
def make_path_file(data,dir,file):
  path1=os.path.join('training',data,dir,file)
  path=PROJECT_PATH+path1
  return path

def calculate_performance(y_pred,test_generator):
  y_pred_bool = np.argmax(y_pred, axis=1)
  print(classification_report(test_generator.labels, y_pred_bool))



def performance_evaluation_model(model,test_generator,verbose = 0):
  y_pred = model.predict_generator(test_generator,verbose = verbose)
  results =model.evaluate(test_generator, batch_size=BATCHSIZE,verbose = verbose)
  # y_pred_bool = np.argmax(y_pred, axis=1)
  print(f'accuracy:{results[1]}, loss: {results[0]}')
  calculate_performance(y_pred,test_generator)
  # print(classification_report(test_generator.labels, y_pred_bool))
  return

# read data

In [ ]:
def read_data(name_file,dir):
  path=make_path_file(DATA,dir,name_file)
  data = pd.read_csv(path)
  y=data['label']
  X=data.drop(['Unnamed: 0','label'], axis=1)
  # mputer = SimpleImputer()
  # X = mputer.fit_transform(x)
  return X.to_numpy(),y.to_numpy()


## train

In [ ]:
train_X,train_y=read_data('train_pred.csv','predictions')

In [ ]:
train_X.shape

(224352, 15)

## test

In [ ]:
test_X,test_y=read_data('test_pred.csv','predictions')

In [ ]:
type(test_X)

numpy.ndarray

# model

In [ ]:
def modelfit(alg, X, y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50,verbose_eval=None):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X, label=y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.n_estimators, nfold=cv_folds,
            metrics=xgb_param['eval_metric'], early_stopping_rounds=early_stopping_rounds,verbose_eval=verbose_eval)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(X,y)

    #Predict training set:
    dtrain_predictions = alg.predict(X)
    # dtrain_predprob = alg.predict_proba(y)[:,1]
    # print(len(dtrain_predictions), len(y))

    #Print model report:
    print( "\nModel Report")
    print (f"Accuracy : {accuracy_score(y, dtrain_predictions)}")
    # print (f"AUC Score (Train): {roc_auc_score(y, dtrain_predprob, multi_class='ovr')}")

    # feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    # feat_imp.plot(kind='bar', title='Feature Importances')
    # plt.ylabel('Feature Importance Score')

## Run

In [ ]:


my_model = XGBClassifier(learning_rate =0.1,
 eval_metric='auc',
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 tree_method='gpu_hist',
 nthread=4,
#  scale_pos_weight=1,
 seed=27,
 num_class=5,
)



In [ ]:
modelfit(my_model, train_X, train_y,verbose_eval=100)

[0]	train-auc:0.99853+0.00002	test-auc:0.99848+0.00008
[100]	train-auc:0.99908+0.00001	test-auc:0.99886+0.00003
[197]	train-auc:0.99936+0.00001	test-auc:0.99887+0.00003

Model Report
Accuracy : 0.976545785194694


## Check Score

In [ ]:
y_test_pred = my_model.predict(test_X)

In [ ]:
score = my_model.score(test_X, test_y)
print("test score: ", score)

test score:  0.9703674648314227


In [ ]:
score = my_model.score(train_X,train_y)
print("Training score: ", score)

Training score:  0.976545785194694


## Save

In [ ]:
path=make_path_file(DATA,'model',"xgb")
my_model.save_model(path)

## Load

In [ ]:
path=make_path_file(DATA,'model',"xgb")
my_model1 = XGBClassifier()
my_model1.load_model(path)

In [ ]:
from sklearn.metrics import classification_report
score = my_model1.score(test_X, test_y)
print("test score: ", score)
pred=my_model1.predict(test_X)
print(classification_report(test_y, pred))


test score:  0.9703674648314227
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15284
           1       0.99      0.99      0.99     10711
           2       0.93      0.92      0.93     10113
           3       0.99      0.99      0.99      9938
           4       0.92      0.94      0.93     10041

    accuracy                           0.97     56087
   macro avg       0.97      0.97      0.97     56087
weighted avg       0.97      0.97      0.97     56087



In [ ]:
from sklearn.metrics import classification_report
pred=my_model1.predict(test_X)
pred[pred > 0] = 1
test_y[test_y>0]=1
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15284
           1       1.00      1.00      1.00     40803

    accuracy                           1.00     56087
   macro avg       1.00      1.00      1.00     56087
weighted avg       1.00      1.00      1.00     56087



# Logistic Regression

In [ ]:
mmodel = LogisticRegression(solver='saga')
mmodel.fit(train_X, train_y)

LogisticRegression(solver='saga')

In [ ]:
score = mmodel.score(train_X,train_y)
print("Training score: ", score)

Training score:  0.9658884253316218


In [ ]:
score = mmodel.score(test_X, test_y)
print("test score: ", score)

test score:  0.9628969279868775


# Soft vote

In [ ]:
X1_test=test_X[:,range(5)]
Y1_test=X1_test.argmax(axis=1)

In [ ]:
accuracy_score(test_y, Y1_test)

0.9640558418171769

In [ ]:
X1_train=train_X[:,range(5)]
Y1_train=X1_train.argmax(axis=1)

In [ ]:
accuracy_score(train_y, Y1_train)

0.9663965554129226

In [ ]:
def get_accuracy(y, pred):
  y_pred=pred.argmax(axis=1)
  return accuracy_score(y, y_pred)

def get_X_for_model(X,i, num_classes):
  start=i*num_classes
  end=start+num_classes
  return X[:,range(start,end)]

def soft_vote(X,y, num_models=3, num_classes=5):
  list=[get_X_for_model(X,i, num_classes) for i in range(num_models)]
  for i,pred in enumerate(list):
    print(f'Accuracy for {i}: {get_accuracy(y, pred)}')
  pred=sum(list)
  print(f'Soft vote accuracy={get_accuracy(y, pred)}')

In [ ]:
soft_vote(test_X,test_y)

Accuracy for 0: 0.9640558418171769
Accuracy for 1: 0.9712945958956621
Accuracy for 2: 0.9639310357123754
Soft vote accuracy=0.9698682404122168


# Hard vote

In [ ]:
def hard_vote(X,y, num_models=3, num_classes=5):
   list=[get_X_for_model(X,i, num_classes) for i in range(num_models)]
   for i,pred in enumerate(list):
     print(f'Accuracy for {i}: {get_accuracy(y, pred)}')
   pred_list=[p.argmax(axis=1) for p in list]
   pred_array=np.vstack(pred_list).transpose()
   pred= mode(pred_array, axis=-1)[0].flatten()
   print(f'Hard vote accuracy={accuracy_score(y, pred)}')
   print(pred.shape,y.shape)
   return pred_array, pred_list


In [ ]:
pred_array, pred_list=hard_vote(test_X,test_y)

Accuracy for 0: 0.9640558418171769
Accuracy for 1: 0.9712945958956621
Accuracy for 2: 0.9639310357123754


<ipython-input-75-200845f35c93>:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  pred= mode(pred_array, axis=-1)[0].flatten()


Hard vote accuracy=0.9671938238807567
(56087,) (56087,)


In [ ]:
idx=15284

In [ ]:
pred_array[idx]

array([1, 1, 1])

In [ ]:
pred_array.shape

(56087, 3)

In [ ]:
[pred_list[i][idx] for i in range(3)]

[1, 1, 1]

In [ ]:
test_y[15284]

1

In [ ]:
np.nonzero(test_y)

(array([15284, 15285, 15286, ..., 56084, 56085, 56086]),)